In [1]:
import numpy as np
import functools
import random
import heapq

def draw_one(s):
    return random.sample(s, 1)[0]


In [20]:
class Chromosome(object):
    
    def __init__(self, num, chromosome, n, fitness_fun, gen):
        self.num = num
        self.chromosome = chromosome
        self.n = n
        self.gen = gen
        self.fitness_fun = fitness_fun
        self.fitness = None
    
    def get_fitness(self):
        if self.fitness == None:
            self.fitness = self.fitness_fun(self.chromosome)
        return self.fitness
    
    def __add__(self, other):
        mutation_prob = gen.mutation_prob
        point = np.random.randint(1, self.n)
        child = self.chromosome[0:point]
        child.extend(other.chromosome[point:])
        p = np.random.random_sample()
        if p <= mutation_prob:
            mutation_point = np.random.randint(0, n)
            old = child[mutation_point]
            alphabet = self.gen.alphabets(mutation_point)
            new = draw_one(alphabet)
            while old == new:
                new = draw_one(alphabet)
            child[mutation_point] = new 
        counter = gen.get_id()
        return Chromosome(counter, child, self.n, self.fitness_fun, self.gen)
    
    def __cmp__(self, other):
        return self.get_fitness() - other.get_fitness()
    
    def __lt__(self, other):
        return self.get_fitness() < other.get_fitness()
    

        
class ChromosomeFactory(object):
    
    def __init__(self, n, fitness_fun, alphabets=(lambda x: [0, 1]), mutation_prob=0.05):
        self.fitness_fun = fitness_fun
        self.alphabets = alphabets
        self.mutation_prob = mutation_prob
        self.n = n
        self.counter = 0
        
    def get_id(self):
        c = self.counter
        self.counter += 1
        return c
    
    def create(self):
        c = []
        for idx in range(self.n):
            alphabet = self.alphabets(idx)
            gene = draw_one(alphabet)
            c.append(gene)
        counter = self.get_id()
        return Chromosome(counter, c, self.n, self.fitness_fun, self)
    
    
        
        

In [14]:
def roulette_wheel_gen(chromosomes):
    ftinesses = map(lambda c: c.get_fitness(), chromosomes)
    fitnesses = np.cumsum(fitnesses) / np.sum(fitnesses)
    def f():
        p = np.random.random_sample()
        idx = 0
        while p > fitnesses[idx + 1]:
            idx += 1
        return chromosomes[idx]
    return f

def get_elites(chromosomes, k=1):
    return heapq.nlargest(k, chromosomes)

def get_parents(chromosomes):
    wheel = roulette_wheel_gen(chromosomes)
    parent1 = wheel()
    parent2 = wheel()
    while parent1.num == parent2.num:
        parent2 = wheel()
    return (parent1, parent2)


    

In [31]:
def fitness(chromosome_as_iterable):
    rep = ''.join(map(str, chromosome_as_iterable))
    x = int(rep, 2)
    return 15 * x - x * x * x
    

In [45]:
def genetic_algorithm(n, fitness_fun, alphabets=(lambda x: [0, 1]), mutation_prob=0.01, crossover_prob=0.8, elitism=2,
                      population_size=20, num_generations=50):
    factory = ChromosomeFactory(n, fitness_fun, alphabets, mutation_prob)
    population = [factory.create() for _ in range(population_size)]
    for generation in range(num_generations):
        new_population = get_elites(population, elitism)
        count = elitism
        for _ in range(count + 1, elitism + 1):
            p = np.random.random_sample()
            parent1, parent2 = get_parents(population)
            if p <= crossover_prob:
                child = parent1 + parent2
                population.append(child)
            else:
                population.append(draw_one([parent1, parent2]))
    return (get_elites(population, 1)[0], population)           
                

In [46]:
solution, population = genetic_algorithm(4, fitness)

In [47]:
solution


In [48]:
solution.chromosome

[0, 0, 1, 1]

In [51]:
population[].chromosome

[0, 1, 0, 0]